# Purpose
This notebook describes the typical activities carried out  at the beginning to a project / thread when customer shares new data. We will be trying to understand the tables, columns and information flow. Typically we also look for data issues and confirm with respective owners for resolution. At the end of this activity, the data sources and their treatment is finalized. Code in this notebook will not be part of the production code.

This data can be downloaded from
[here](https://drive.google.com/file/d/11DqcBxxEcn3QA4YvPQmmExBm-m6AgUQ_/view?usp=sharing)

**NOTE**:
Download the data from the above link, and copy the extracted csv files to the path `data/raw/sales/` (relative to root of the code archive folder). Make sure to copy the files before continuing on with the rest of the notebook.

# Initialization

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
%%time

# Third-party imports
import os.path as op
import pandas as pd
import great_expectations as ge
import numpy as np
import matplotlib
matplotlib.use('TkAgg')

import matplotlib.pyplot as plt


# Project imports
from ta_lib.core.api import display_as_tabs, initialize_environment

# Initialization
initialize_environment(debug=False, hide_warnings=True)

CPU times: total: 93.8 ms
Wall time: 71.1 ms


# Data

## Background

Customer is a distributor of electronic devices. They partner with manufacturers, carriers and refurbishers and sell across to  retailers. The selling price is the outcome of negotiation between sales representatives and retailers. Customer wants to understand the selling price variation and determine  optimal pricing with Machine Learning.

In [5]:
from ta_lib.core.api import create_context, list_datasets, load_dataset

In [6]:
config_path = op.join('conf', 'config.yml')
context = create_context(config_path)

In [7]:
list_datasets(context)

['/raw/google_search_data',
 '/raw/product_manufacturer_list',
 '/raw/sales_data',
 '/raw/social_media_data',
 '/raw/Theme_list',
 '/raw/Theme_product_list']

In [8]:
# load datasets
google_search_data = load_dataset(context, 'raw/google_search_data')
product_manufacturer_list = load_dataset(context, 'raw/product_manufacturer_list')
sales_data = load_dataset(context, 'raw/sales_data')
social_media_data = load_dataset(context, 'raw/social_media_data')
Theme_list = load_dataset(context, 'raw/Theme_list')
Theme_product_list = load_dataset(context, 'raw/Theme_product_list')

In [9]:
dataset_name = ['google_search_data', 'product_manufacturer_list', 'sales_data', 'social_media_data', 'Theme_list', 'Theme_product_list']
dataset = [google_search_data, product_manufacturer_list, sales_data, social_media_data, Theme_list, Theme_product_list]

# Exploratory Analysis

Given the raw data from data ingestion, we would now like to explore and learn more details about the data.


The output of the step would be a summary report and discussion of any pertinent findings.


In [10]:
# Import the eda API
import ta_lib.eda.api as eda

## Variable summary

In [11]:
display_as_tabs([('google_search_data', google_search_data.shape), 
                 ('product_manufacturer_list', product_manufacturer_list.shape),
                 ('sales_data', sales_data.shape),
                 ('social_media_data', social_media_data.shape),
                 ('Theme_list', Theme_list.shape),
                 ('Theme_product_list', Theme_product_list.shape)                
                ])

Tabs
    [0] Str(tuple, name='google_search_data')
    [1] Str(tuple, name='product_manufacturer_list...)
    [2] Str(tuple, name='sales_data')
    [3] Str(tuple, name='social_media_data')
    [4] Str(tuple, name='Theme_list')
    [5] Str(tuple, name='Theme_product_list')

In [12]:
sum1 = eda.get_variable_summary(social_media_data)
sum2 = eda.get_variable_summary(google_search_data)
sum3 = eda.get_variable_summary(sales_data)


display_as_tabs([('orders', sum1), ('product', sum2), ('sales', sum3)])

Tabs
    [0] DataFrame(height=300, name='orders', sizing_mode='fixed', value=                 Variable ..., width=800)
    [1] DataFrame(height=300, name='product', sizing_mode='fixed', value=             Variable Name..., width=800)
    [2] DataFrame(height=300, name='sales', sizing_mode='fixed', value=                          ..., width=800)

In [13]:
import os
print(os.listdir())

['.ipynb_checkpoints', '01_data_discovery.ipynb', '02_data_processing.ipynb', '03_model_experimenting.ipynb', 'conf', 'logs', 'scripts.py', 'social_media_data.xlsx', 'u3.xlsx']


In [14]:
# read_file = pd.read_excel("social_media_data.xlsx")
# read_file.isna().sum()

In [15]:
for i in range(len(dataset)):
    print(dataset_name[i])
    print(dataset[i].isna().sum())
    print('-----------------------------')

google_search_data
date            0
platform        0
searchVolume    0
Claim_ID        0
week_number     0
year_new        0
dtype: int64
-----------------------------
product_manufacturer_list
PRODUCT_ID        0
Vendor            0
Unnamed: 2    67175
Unnamed: 3    67175
Unnamed: 4    67175
Unnamed: 5    67175
Unnamed: 6    67175
dtype: int64
-----------------------------
sales_data
system_calendar_key_N    0
product_id               0
sales_dollars_value      0
sales_units_value        0
sales_lbs_value          0
dtype: int64
-----------------------------
social_media_data
Theme Id          218511
published_date         0
total_post             0
dtype: int64
-----------------------------
Theme_list
CLAIM_ID      0
Claim Name    0
dtype: int64
-----------------------------
Theme_product_list
PRODUCT_ID    0
CLAIM_ID      0
dtype: int64
-----------------------------


In [16]:
product_manufacturer_list.dropna(axis=1, inplace=True)
social_media_data.dropna(inplace=True)

In [17]:
# read_file.shape
c_social_media = pd.merge(social_media_data, Theme_list, left_on='Theme Id', right_on='CLAIM_ID', how = 'inner')
c_google = pd.merge(google_search_data, Theme_list, left_on='Claim_ID', right_on='CLAIM_ID', how = 'inner')
c_sales = pd.merge(sales_data, Theme_product_list, left_on='product_id', right_on='PRODUCT_ID', how = 'inner')
c_sales = pd.merge(c_sales, Theme_list, on='CLAIM_ID', how = 'inner')

In [18]:
data_source = [c_social_media, c_google, c_sales]
data_source_names = ['Social Media Data', 'Google Search Data', 'Sales Data']

In [19]:
# ●	Provide the list of themes available across all data sources
for i in range(3):
    print(data_source_names[i])
    themes = data_source[i]['Claim Name'].unique()
    print(themes)
    print('Total Themes : ', len(themes)) 
    print('-------------------------------------')

Social Media Data
['tuna' 'allergy' 'convenience - packaging' 'oral health' 'mint' 'dry'
 'traditional' 'antioxidant' 'peppermint' 'skin health' 'banana' 'crab'
 'mango' 'peanut butter' 'immune health' 'broccoli' 'spinach'
 'convenience - ready prepared' 'toffee' 'indulgent & premium' 'organic'
 'low fat' 'vitamin/mineral fortified' 'trout' 'low calorie' 'bacon'
 'coconut' 'red apple' 'high/source of fiber' 'low sodium'
 'no added sugar' 'green bean' 'omega-3' 'bone health' 'green tea'
 'cranberry' 'heart health' 'weight management' 'celery' 'strawberry'
 'dandelion greens' 'prebiotic' 'ethical - packaging' 'eye health'
 'poultry' 'parsley' 'economy' 'cheddar cheese' 'venison'
 'ethical - animal/fish & bird' 'low sugar' 'ricotta cheese'
 'joint health' 'brain health' 'roquefort cheese' 'no trans fats'
 'sugar free' 'diabetic' 'hfcs free' 'low cholesterol' 'kale' 'smoked'
 'low gi' 'cashew nut' 'pheasant' 'chili' 'mackerel' 'herring' 'orange'
 'lactose free' 'wholegrain' 'added calcium'

NameError: name 'em_themes' is not defined

In [20]:
# ●	Understands consumer preference(themes) available in each data source
factor = ['total_post', 'searchVolume', 'sales_units_value']
for i in range(3):
    print(data_source_names[i])
    print(data_source[i].groupby('Claim Name').sum()[factor[i]].reset_index().sort_values(factor[i], ascending=False))
    print('--------------------------------')


Social Media Data
           Claim Name  total_post
84   health (passive)     5329592
18               boar     4609405
146            rabbit     2821690
143         probiotic     2462718
144           pumpkin     2417031
..                ...         ...
24   butternut squash          29
161           seaweed          22
104           lobster          10
73              fruit           3
151    ricotta cheese           0

[193 rows x 2 columns]
--------------------------------
Google Search Data
                Claim Name  searchVolume
50   ethical - environment     113502487
133                 shrimp      77711760
144             sugar free      74588723
77                   honey      73957180
68        health (passive)      56756609
..                     ...           ...
136                 smoked            11
16                  butter             5
17                   carob             3
19              cashew nut             3
157   white cheddar cheese             3

[160 

In [30]:
em_sales = c_sales.groupby('Claim Name').sum()[factor[i]].reset_index().sort_values(factor[i], ascending=False)
di = dict(zip(em_sales['Claim Name'], em_sales['sales_units_value']))
len(di)
for i in range(4):
    for j in range(len(di)/4):
        di[]
# themes_sales = np.array(em_sales['sales_units_value'])
# themes_sales

49

In [42]:
part_len = len(di)//4
c=0
part = 1
for key in di:
    c += 1
    di[key] = part
    if(c>=part_len):
        c=0
        part += 1

di['tuna'] = 4
di

{'No Claim': 1,
 'low carb': 1,
 'no additives/preservatives': 1,
 'stroganoff': 1,
 'apple cinnamon': 1,
 'soy foods': 1,
 'pollock': 1,
 'gmo free': 1,
 'buckwheat': 1,
 'salmon': 1,
 'cookie': 1,
 'pizza': 1,
 'ethnic & exotic': 2,
 'american southwest style': 2,
 'blueberry': 2,
 'french bisque': 2,
 'chicken': 2,
 'crab': 2,
 'low sodium': 2,
 'american gumbo': 2,
 'brown ale': 2,
 'low calorie': 2,
 'beans': 2,
 'herbs': 2,
 'gingerbread': 3,
 'high/source of protein': 3,
 'cocoa': 3,
 'low sugar': 3,
 'ethical - packaging': 3,
 'vegetarian': 3,
 'scallop': 3,
 'red raspberry': 3,
 'poultry': 3,
 'tilapia': 3,
 'nuts': 3,
 'energy/alertness': 3,
 'prebiotic': 4,
 'convenience - easy-to-prepare': 4,
 'beef hamburger': 4,
 'peach': 4,
 'peanut': 4,
 'cherry': 4,
 'halal': 4,
 'bone health': 4,
 'mackerel': 4,
 'cumin': 4,
 'ethical - not specific': 4,
 'sea salt': 4,
 'tuna': 4}

In [44]:
c_sales = c_sales.replace({"Claim Name": di})

In [46]:
Vendor_dummies = pd.get_dummies(c_sales['Claim Name'], drop_first=True)
df2 = pd.concat([c_sales, Vendor_dummies], axis=1)
df2

,system_calendar_key_N,product_id,sales_dollars_value,sales_units_value,sales_lbs_value,PRODUCT_ID,CLAIM_ID,Claim Name,2,3,4
0,20160109,1,13927.0,934,18680,1,0,1,0,0,0
1,20160123,1,12628.0,878,17564,1,0,1,0,0,0
2,20160206,1,11379.0,810,16200,1,0,1,0,0,0
3,20160130,1,11568.0,821,16424,1,0,1,0,0,0
4,20160213,1,10959.0,784,15682,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
7767415,20190907,18405,2313.0,91,273,18405,65,4,0,0,1
7767416,20190914,18405,2313.0,91,273,18405,65,4,0,0,1
7767417,20190921,18405,2313.0,91,273,18405,65,4,0,0,1
7767418,20190928,18405,2313.0,91,273,18405,65,4,0,0,1


In [65]:
df2 = df2.sample(frac = 1)
df2

,system_calendar_key_N,product_id,sales_dollars_value,sales_units_value,sales_lbs_value,PRODUCT_ID,CLAIM_ID,Claim Name,2,3,4
885530,20190921,13901,7511.0,8378,1832,13901,0,1,0,0,0
6712094,20190202,31294,3282.0,481,75,31294,8,1,0,0,0
800879,20180317,16683,8892.0,6831,2348,16683,0,1,0,0,0
3139280,20160123,33074,4179.0,115,1153,33074,222,1,0,0,0
7287324,20190601,39493,9830.0,402,1608,39493,81,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
3528552,20180818,22747,150833.0,4955,74325,22747,187,2,1,0,0
7546238,20170506,6208,1256.0,162,36,6208,207,3,0,1,0
4483664,20190622,23920,5631.0,4436,832,23920,40,1,0,0,0
4695568,20160305,28127,12525.0,216,4752,28127,40,1,0,0,0


In [66]:
X = df2[[2,3,4]]
y = df2['sales_dollars_value']

In [67]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [68]:
y_pred = regressor.predict(X_test)
print(y_pred)

[20890.35111125 20890.35111125 20890.35111125 ... 20890.35111125
 20890.35111125 20890.35111125]


In [69]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
# predicting the accuracy score
score=r2_score(y_test,y_pred)
print('r2 socre is ',score)
print('mean_sqrd_error is==',mean_squared_error(y_test,y_pred))
print('root_mean_squared error of is==',np.sqrt(mean_squared_error(y_test,y_pred)))

r2 socre is  0.0024725997563675994
mean_sqrd_error is== 4593828261.610302
root_mean_squared error of is== 67777.78590076767


In [28]:
l = np.array_split(np.array(themes_sales),4)
l

[array([18581260352,  2627421936,  1559993572,  1391218733,   805116403,
          449104186,   378731037,   327952459,   307259973,   288110889,
          267479854,   267479854,   135668268], dtype=int64),
 array([125512861, 125512861,  82170776,  69625831,  41664448,  35262508,
         20572186,  19890723,  18261984,  17156758,  14261678,  13898262],
       dtype=int64),
 array([13695191, 12968838,  9334803,  7014742,  6182792,  4274509,
         4274058,  4010072,  3481869,  2687280,  2169678,  1642137],
       dtype=int64),
 array([1135976,  903427,  497171,  425440,  354199,  181528,  103894,
          35791,    5732,    5686,    3123,      72], dtype=int64)]

In [19]:
# read_file.shape

**Dev NOTES**

<details>
1. Datatypes : We have both numeric and other types. The bulk of them seem to be numeric. `Numeric` is defined to be one of [float|int|date] and the rest are categorized as `Others`. A column is assumed to have `date` values if it has the string `date` in the column name.


## Merging

We can merge orders table with prod table on SKU. Let us check first-cut cardinality issues. 

### Expected data validation rules

1. Quantity should be an integer
2. Quantity * UnitCost = SellingCost
3. Quantity * UnitPrice = SellingPrice

In [20]:
vendor = ['A', 'B', 'D', 'E', 'F', 'G', 'H', 'Others', 'Private Label']

In [21]:
# product_manufacturer_list[product_manufacturer_list['Vendor'] == 'A']
vendor_dataset = []
for v in vendor:
    vendor_dataset.append(product_manufacturer_list[product_manufacturer_list['Vendor'] == v])

vendor_dataset[0]

,PRODUCT_ID,Vendor
571,572,A
572,573,A
573,574,A
574,575,A
575,576,A
...,...,...
57418,57419,A
57419,57420,A
57420,57421,A
57425,57426,A


In [22]:
# uu = pd.merge(vendor_dataset[0], Theme_product_list, on='PRODUCT_ID', how='inner')
# uu = pd.merge(uu, Theme_list, on='CLAIM_ID', how='inner')
# uu = uu[uu['CLAIM_ID'] != 0]
# uu = pd.merge(uu, social_media_data, left_on='CLAIM_ID', right_on='Theme Id', how='inner')
# # uu = uu.drop_duplicate_columns()

# # uu = pd.merge(uu, google_search_data, left_on='CLAIM_ID', right_on='Claim_ID', how='inner')
# # uu = pd.merge(uu, sales_data, left_on='PRODUCT_ID', right_on='product_id', how='inner')
# print(uu.isna().sum())
# uu

In [23]:
# uu = uu.groupby(['PRODUCT_ID', 'CLAIM_ID'])['total_post'].sum().reset_index().sort_values('PRODUCT_ID')

u1 = social_media_data.groupby('Theme Id')['total_post'].sum().reset_index()
u1 = pd.merge(u1, Theme_product_list, left_on='Theme Id', right_on='CLAIM_ID', how='inner')
u1

,Theme Id,total_post,PRODUCT_ID,CLAIM_ID
0,8.0,535903,26,8
1,8.0,535903,29,8
2,8.0,535903,86,8
3,8.0,535903,87,8
4,8.0,535903,120,8
...,...,...,...,...
40273,438.0,527730,40474,438
40274,438.0,527730,41143,438
40275,438.0,527730,41250,438
40276,438.0,527730,55600,438


In [24]:
ee = pd.merge(social_media_data, Theme_product_list, left_on='Theme Id', right_on='CLAIM_ID', how='inner')

In [25]:
ee.groupby(['PRODUCT_ID', 'CLAIM_ID'])['total_post'].sum().reset_index().sort_values('total_post')

,PRODUCT_ID,CLAIM_ID,total_post
13758,19683,385,582
19750,26664,385,582
19744,26663,385,582
19740,26662,385,582
19736,26661,385,582
...,...,...,...
11239,17664,142,867086
11240,17666,142,867086
11241,17667,142,867086
14535,21174,142,867086


In [26]:
# u1.sort_values('total_post')
# u1[u1['PRODUCT_ID'] == 26 & u1['CLAIM_ID'] == 8]

In [27]:
u2 = google_search_data.groupby('Claim_ID')['searchVolume'].sum().reset_index()
u2 = pd.merge(u2, Theme_product_list, left_on='Claim_ID', right_on='CLAIM_ID', how='inner')
u2

,Claim_ID,searchVolume,PRODUCT_ID,CLAIM_ID
0,8,14911004,26,8
1,8,14911004,29,8
2,8,14911004,86,8
3,8,14911004,87,8
4,8,14911004,120,8
...,...,...,...,...
39326,438,24798675,40474,438
39327,438,24798675,41143,438
39328,438,24798675,41250,438
39329,438,24798675,55600,438


In [28]:
u3 = sales_data.groupby('product_id')['sales_dollars_value', 'sales_units_value', 'sales_lbs_value'].sum().reset_index()
u3 = pd.merge(u3, Theme_product_list, left_on='product_id', right_on='PRODUCT_ID', how='inner')
u3

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,product_id,sales_dollars_value,sales_units_value,sales_lbs_value,PRODUCT_ID,CLAIM_ID
0,1,2403531.0,178483,3569652,1,0
1,2,3841.0,281,11248,2,0
2,3,1615770.0,247684,4458341,3,0
3,4,165975.0,14280,285617,4,0
4,5,25312.0,1520,60942,5,0
...,...,...,...,...,...,...
70334,57314,13305.0,2523,2169,57314,40
70335,57314,13305.0,2523,2169,57314,8
70336,57315,1492.0,633,118,57315,8
70337,57316,3466.0,2082,390,57316,8


In [29]:
u1 = pd.merge(u1, u2, left_on=['Theme Id', 'PRODUCT_ID'], right_on=['CLAIM_ID', 'PRODUCT_ID'], how='inner')
u1

,Theme Id,total_post,PRODUCT_ID,CLAIM_ID_x,Claim_ID,searchVolume,CLAIM_ID_y
0,8.0,535903,26,8,8,14911004,8
1,8.0,535903,29,8,8,14911004,8
2,8.0,535903,86,8,8,14911004,8
3,8.0,535903,87,8,8,14911004,8
4,8.0,535903,120,8,8,14911004,8
...,...,...,...,...,...,...,...
39326,438.0,527730,40474,438,438,24798675,438
39327,438.0,527730,41143,438,438,24798675,438
39328,438.0,527730,41250,438,438,24798675,438
39329,438.0,527730,55600,438,438,24798675,438


In [30]:
u1 = pd.merge(u1, u3, left_on=['Theme Id', 'PRODUCT_ID'], right_on=['CLAIM_ID', 'PRODUCT_ID'], how='inner')
# u1.drop(['Theme Id', 'CLAIM_ID_x', 'Claim_ID', 'CLAIM_ID_y', 'product_id'], axis=1, inplace=True)
u1

,Theme Id,total_post,PRODUCT_ID,CLAIM_ID_x,Claim_ID,searchVolume,CLAIM_ID_y,product_id,sales_dollars_value,sales_units_value,sales_lbs_value,CLAIM_ID
0,8.0,535903,26,8,8,14911004,8,26,472.0,39,77,8
1,8.0,535903,29,8,8,14911004,8,29,144.0,121,35,8
2,8.0,535903,120,8,8,14911004,8,120,2.0,1,0,8
3,8.0,535903,278,8,8,14911004,8,278,850.0,53,175,8
4,8.0,535903,470,8,8,14911004,8,470,4489.0,379,141,8
...,...,...,...,...,...,...,...,...,...,...,...,...
32064,438.0,527730,40474,438,438,24798675,438,40474,560570.0,22715,340725,438
32065,438.0,527730,41143,438,438,24798675,438,41143,98060664.0,1929163,57875067,438
32066,438.0,527730,41250,438,438,24798675,438,41250,503812.0,9768,293226,438
32067,438.0,527730,55600,438,438,24798675,438,55600,6035.0,217,175,438


In [31]:
u1 = pd.merge(u1, product_manufacturer_list, on='PRODUCT_ID', how='inner')
u1

,Theme Id,total_post,PRODUCT_ID,CLAIM_ID_x,Claim_ID,searchVolume,CLAIM_ID_y,product_id,sales_dollars_value,sales_units_value,sales_lbs_value,CLAIM_ID,Vendor
0,8.0,535903,26,8,8,14911004,8,26,472.0,39,77,8,Others
1,227.0,231258,26,227,227,1531221,227,26,472.0,39,77,227,Others
2,8.0,535903,29,8,8,14911004,8,29,144.0,121,35,8,Others
3,8.0,535903,120,8,8,14911004,8,120,2.0,1,0,8,Others
4,8.0,535903,278,8,8,14911004,8,278,850.0,53,175,8,Others
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32064,438.0,527730,37509,438,438,24798675,438,37509,735.0,158,42,438,Others
32065,438.0,527730,44609,438,438,24798675,438,44609,162774.0,14087,70436,438,Private Label
32066,438.0,527730,35576,438,438,24798675,438,35576,130326.0,2047,53217,438,Others
32067,438.0,527730,39656,438,438,24798675,438,39656,45248.0,32470,4068,438,Others


In [32]:
sales_data.shape

(4526182, 5)

In [33]:
u1

,Theme Id,total_post,PRODUCT_ID,CLAIM_ID_x,Claim_ID,searchVolume,CLAIM_ID_y,product_id,sales_dollars_value,sales_units_value,sales_lbs_value,CLAIM_ID,Vendor
0,8.0,535903,26,8,8,14911004,8,26,472.0,39,77,8,Others
1,227.0,231258,26,227,227,1531221,227,26,472.0,39,77,227,Others
2,8.0,535903,29,8,8,14911004,8,29,144.0,121,35,8,Others
3,8.0,535903,120,8,8,14911004,8,120,2.0,1,0,8,Others
4,8.0,535903,278,8,8,14911004,8,278,850.0,53,175,8,Others
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32064,438.0,527730,37509,438,438,24798675,438,37509,735.0,158,42,438,Others
32065,438.0,527730,44609,438,438,24798675,438,44609,162774.0,14087,70436,438,Private Label
32066,438.0,527730,35576,438,438,24798675,438,35576,130326.0,2047,53217,438,Others
32067,438.0,527730,39656,438,438,24798675,438,39656,45248.0,32470,4068,438,Others


In [34]:
vendor_sales = u1.groupby('Vendor')['sales_units_value'].sum().reset_index().sort_values('sales_units_value', ascending = False)

In [35]:
s = np.array(vendor_sales['sales_units_value'])
print(s[1]/s.sum())

0.23238184125186714


In [36]:
px = pd.merge(sales_data, product_manufacturer_list, left_on='product_id', right_on='PRODUCT_ID', how='inner')
px

,system_calendar_key_N,product_id,sales_dollars_value,sales_units_value,sales_lbs_value,PRODUCT_ID,Vendor
0,20160109,1,13927.0,934,18680,1,Others
1,20160123,1,12628.0,878,17564,1,Others
2,20160206,1,11379.0,810,16200,1,Others
3,20160130,1,11568.0,821,16424,1,Others
4,20160213,1,10959.0,784,15682,1,Others
...,...,...,...,...,...,...,...
4526177,20191005,56817,699.0,37,129,56817,Others
4526178,20191005,56818,486.0,7,158,56818,Others
4526179,20191005,56924,53.0,3,16,56924,Others
4526180,20191005,57045,6.0,1,7,57045,Private Label


In [37]:
prod = pd.merge(google_search_data, Theme_list, left_on='Claim_ID', right_on='CLAIM_ID', how = 'inner')
prod

,date,platform,searchVolume,Claim_ID,week_number,year_new,CLAIM_ID,Claim Name
0,2014-01-05,google,349,916,1,2014,916,gluten free
1,2014-01-06,google,349,916,2,2014,916,gluten free
2,2014-01-07,google,697,916,2,2014,916,gluten free
3,2014-01-10,google,349,916,2,2014,916,gluten free
4,2014-01-20,google,697,916,4,2014,916,gluten free
...,...,...,...,...,...,...,...,...
181560,2019-07-24,google,42,843,30,2019,843,snickerdoodle
181561,2019-09-25,google,84,843,39,2019,843,snickerdoodle
181562,2018-06-04,amazon,135,606,23,2018,606,hemp seeds
181563,2019-01-31,amazon,39,433,5,2019,433,gingerbread


In [38]:
vendor_sales = px.groupby('Vendor')['sales_dollars_value'].sum().reset_index().sort_values('sales_dollars_value', ascending = False)
vendor_sales

,Vendor,sales_dollars_value
0,A,3.093204e+10
7,Others,1.854419e+10
1,B,1.538074e+10
2,D,1.196683e+10
8,Private Label,9.753070e+09
4,F,4.995877e+09
6,H,2.277657e+09
5,G,1.951155e+09
3,E,1.939263e+09


In [39]:
s = np.array(vendor_sales['sales_dollars_value'])
print(s)
print(s[0]/s.sum())

[3.09320438e+10 1.85441857e+10 1.53807445e+10 1.19668273e+10
 9.75306998e+09 4.99587718e+09 2.27765658e+09 1.95115487e+09
 1.93926335e+09]
0.3164700563323608


In [40]:
vendor_sales['Vendor']

0                A
7           Others
1                B
2                D
8    Private Label
4                F
6                H
5                G
3                E
Name: Vendor, dtype: object

In [41]:
mylabels = np.array(vendor_sales['Vendor'])
print(mylabels)
plt.pie(s, labels = mylabels, autopct='%1.1f%%')
plt.legend(title = "Market Share")
plt.show() 

Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.



['A' 'Others' 'B' 'D' 'Private Label' 'F' 'H' 'G' 'E']


In [42]:
u1.corr()

,Theme Id,total_post,PRODUCT_ID,CLAIM_ID_x,Claim_ID,searchVolume,CLAIM_ID_y,product_id,sales_dollars_value,sales_units_value,sales_lbs_value,CLAIM_ID
Theme Id,1.000000,-0.265650,-0.172435,1.000000,1.000000,0.019629,1.000000,-0.172435,0.047981,0.018101,0.067527,1.000000
total_post,-0.265650,1.000000,0.091511,-0.265650,-0.265650,0.775521,-0.265650,0.091511,-0.047189,-0.015411,-0.039003,-0.265650
PRODUCT_ID,-0.172435,0.091511,1.000000,-0.172435,-0.172435,0.063136,-0.172435,1.000000,-0.034079,-0.036289,-0.037326,-0.172435
CLAIM_ID_x,1.000000,-0.265650,-0.172435,1.000000,1.000000,0.019629,1.000000,-0.172435,0.047981,0.018101,0.067527,1.000000
Claim_ID,1.000000,-0.265650,-0.172435,1.000000,1.000000,0.019629,1.000000,-0.172435,0.047981,0.018101,0.067527,1.000000
searchVolume,0.019629,0.775521,0.063136,0.019629,0.019629,1.000000,0.019629,0.063136,-0.050215,-0.033545,-0.040689,0.019629
CLAIM_ID_y,1.000000,-0.265650,-0.172435,1.000000,1.000000,0.019629,1.000000,-0.172435,0.047981,0.018101,0.067527,1.000000
product_id,-0.172435,0.091511,1.000000,-0.172435,-0.172435,0.063136,-0.172435,1.000000,-0.034079,-0.036289,-0.037326,-0.172435
sales_dollars_value,0.047981,-0.047189,-0.034079,0.047981,0.047981,-0.050215,0.047981,-0.034079,1.000000,0.443072,0.885743,0.047981
sales_units_value,0.018101,-0.015411,-0.036289,0.018101,0.018101,-0.033545,0.018101,-0.036289,0.443072,1.000000,0.384287,0.018101


In [43]:
sales_data.corr()

,system_calendar_key_N,product_id,sales_dollars_value,sales_units_value,sales_lbs_value
system_calendar_key_N,1.000000,0.010508,-0.003643,-0.010182,-0.006581
product_id,0.010508,1.000000,-0.147133,-0.125709,-0.074513
sales_dollars_value,-0.003643,-0.147133,1.000000,0.391578,0.837277
sales_units_value,-0.010182,-0.125709,0.391578,1.000000,0.227294
sales_lbs_value,-0.006581,-0.074513,0.837277,0.227294,1.000000


In [44]:
Vendor_dummies = pd.get_dummies(u1['Vendor'], drop_first=True)
df2 = pd.concat([u1, Vendor_dummies], axis=1)
df2

,Theme Id,total_post,PRODUCT_ID,CLAIM_ID_x,Claim_ID,searchVolume,CLAIM_ID_y,product_id,sales_dollars_value,sales_units_value,sales_lbs_value,CLAIM_ID,Vendor,B,D,F,H,Others,Private Label
0,8.0,535903,26,8,8,14911004,8,26,472.0,39,77,8,Others,0,0,0,0,1,0
1,227.0,231258,26,227,227,1531221,227,26,472.0,39,77,227,Others,0,0,0,0,1,0
2,8.0,535903,29,8,8,14911004,8,29,144.0,121,35,8,Others,0,0,0,0,1,0
3,8.0,535903,120,8,8,14911004,8,120,2.0,1,0,8,Others,0,0,0,0,1,0
4,8.0,535903,278,8,8,14911004,8,278,850.0,53,175,8,Others,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32064,438.0,527730,37509,438,438,24798675,438,37509,735.0,158,42,438,Others,0,0,0,0,1,0
32065,438.0,527730,44609,438,438,24798675,438,44609,162774.0,14087,70436,438,Private Label,0,0,0,0,0,1
32066,438.0,527730,35576,438,438,24798675,438,35576,130326.0,2047,53217,438,Others,0,0,0,0,1,0
32067,438.0,527730,39656,438,438,24798675,438,39656,45248.0,32470,4068,438,Others,0,0,0,0,1,0


### Modelling

In [45]:
X = df2[['searchVolume', 'sales_lbs_value', 'sales_units_value', 'B', 'D', 'F', 'H', 'Others', 'Private Label']]
y = df2['sales_dollars_value']

In [231]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [233]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [234]:
y_pred = regressor.predict(X_test)
print(y_pred)
# np.set_printoptions(precision=2)
# print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[ 834706.81  493684.32  747029.39 ...  536347.79 2374429.72  544918.27]


In [191]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
# predicting the accuracy score
score=r2_score(y_test,y_pred)
print('r2 socre is ',score)
print('mean_sqrd_error is==',mean_squared_error(y_test,y_pred))
print('root_mean_squared error of is==',np.sqrt(mean_squared_error(y_test,y_pred)))

r2 socre is  0.7067165687284918
mean_sqrd_error is== 8589648185935.671
root_mean_squared error of is== 2930810.158631171


In [210]:
sales_data.corr()

,system_calendar_key_N,product_id,sales_dollars_value,sales_units_value,sales_lbs_value
system_calendar_key_N,1.000000,0.010508,-0.003643,-0.010182,-0.006581
product_id,0.010508,1.000000,-0.147133,-0.125709,-0.074513
sales_dollars_value,-0.003643,-0.147133,1.000000,0.391578,0.837277
sales_units_value,-0.010182,-0.125709,0.391578,1.000000,0.227294
sales_lbs_value,-0.006581,-0.074513,0.837277,0.227294,1.000000


In [146]:
plat = ['google', 'amazon', 'walmart', 'chewy']

In [68]:
for p in plat:
    print(p)
    df = prod[prod['platform'] == p]
    df = df.groupby('Claim Name').sum()['searchVolume'].reset_index()
    print(df.sort_values('searchVolume', ascending=False))
    print("------------------------------")
    

google
                Claim Name  searchVolume
50   ethical - environment     113482508
132                 shrimp      77498586
143             sugar free      74104963
76                   honey      73600640
68        health (passive)      54400741
..                     ...           ...
135                 smoked            11
16                  butter             5
17                   carob             3
19              cashew nut             3
156   white cheddar cheese             3

[159 rows x 2 columns]
------------------------------
amazon
            Claim Name  searchVolume
48    health (passive)       1409889
10                boar       1000263
60            low carb        893594
54           hfcs free        608938
11         bone health        575161
..                 ...           ...
74     parmesan cheese            75
106  weight management            71
43         gingerbread            39
45          green bean            32
70             omega-3          

In [51]:
df = google_search_data[google_search_data['platform'] == 'google']

In [56]:
df = df.groupby('Claim_ID').sum()['searchVolume'].reset_index()

In [59]:
df.sort_values('searchVolume', ascending=False)

,Claim_ID,searchVolume
117,869,113482508
146,964,77498586
61,613,74104963
153,980,73600640
53,576,54400741
...,...,...
102,799,11
140,941,5
75,681,3
39,514,3


In [17]:
# applying merge with more parameters
# df = pd.merge(a, b, on='id', how='inner')
prod = pd.merge(product_manufacturer_list, Theme_product_list, on='PRODUCT_ID', how = 'inner')
print(prod)
# product_manufacturer_list.merge(Theme_product_list, on='PRODUCT_ID', how = 'outer')
# product_manufacturer_list

       PRODUCT_ID  Vendor  CLAIM_ID
0               1  Others         0
1               2  Others         0
2               3  Others         0
3               4  Others         0
4               5  Others         0
...           ...     ...       ...
91480       57314  Others        40
91481       57314  Others         8
91482       57315  Others         8
91483       57316  Others         8
91484       57317  Others         8

[91485 rows x 3 columns]


In [18]:
product_manufacturer_list.head()

,PRODUCT_ID,Vendor
0,1,Others
1,2,Others
2,3,Others
3,4,Others
4,5,Others


In [19]:
prod = pd.merge(prod, sales_data, left_on='PRODUCT_ID', right_on='product_id', how = 'inner')
print(prod)
# product_manufacturer_list.merge(sales_data, left_on='PRODUCT_ID', right_on='product_id', how = 'inner')

         PRODUCT_ID  Vendor  CLAIM_ID  system_calendar_key_N  product_id  \
0                 1  Others         0               20160109           1   
1                 1  Others         0               20160123           1   
2                 1  Others         0               20160206           1   
3                 1  Others         0               20160130           1   
4                 1  Others         0               20160213           1   
...             ...     ...       ...                    ...         ...   
7767415       57317  Others         8               20190907       57317   
7767416       57317  Others         8               20190914       57317   
7767417       57317  Others         8               20190921       57317   
7767418       57317  Others         8               20190928       57317   
7767419       57317  Others         8               20191005       57317   

         sales_dollars_value  sales_units_value  sales_lbs_value  
0                   

In [30]:
them = pd.merge(google_search_data, social_media_data, left_on=['Claim_ID', 'date'], right_on=['Theme Id', 'published_date'], how='inner')

In [31]:
them

,date,platform,searchVolume,Claim_ID,week_number,year_new,Theme Id,published_date,total_post
0,08-02-2015,google,349,916,6,2015,916.0,08-02-2015,4
1,09-02-2015,google,349,916,7,2015,916.0,09-02-2015,23
2,07-03-2015,google,349,916,10,2015,916.0,07-03-2015,3
3,10-03-2015,google,1046,916,11,2015,916.0,10-03-2015,14
4,11-03-2015,google,697,916,11,2015,916.0,11-03-2015,30
...,...,...,...,...,...,...,...,...,...
58980,06-08-2019,google,3674,536,32,2019,536.0,06-08-2019,0
58981,10-08-2019,google,3674,536,32,2019,536.0,10-08-2019,0
58982,02-09-2019,google,7386,536,36,2019,536.0,02-09-2019,1
58983,03-09-2019,google,10617,536,36,2019,536.0,03-09-2019,3


In [32]:
sales_data

,system_calendar_key_N,product_id,sales_dollars_value,sales_units_value,sales_lbs_value
0,20160109,1,13927.0,934,18680
1,20160109,3,10289.0,1592,28646
2,20160109,4,357.0,22,440
3,20160109,6,23113.0,2027,81088
4,20160109,7,23177.0,3231,58164
...,...,...,...,...,...
4526177,20181027,47536,8.0,2,3
4526178,20181027,47539,391.0,39,68
4526179,20181027,47543,105.0,59,48
4526180,20181027,47544,3720.0,1246,4361


In [24]:
social_media_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 314879 entries, 0 to 533389
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Theme Id        314879 non-null  float64
 1   published_date  314879 non-null  object 
 2   total_post      314879 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 9.6+ MB


In [ ]:
them.drop('Theme Id')

In [21]:
them = pd.merge(them, Theme_list, left_on='Claim_ID', right_on='CLAIM_ID', how='inner')

MemoryError: Unable to allocate 2.24 GiB for an array with shape (300278636,) and data type int64

In [11]:
verification_dict = {}
orders_df = ge.from_pandas(orders_df)

#### Rule 1 verification

In [12]:
verification_dict["rule_1_check"] = orders_df.expect_column_values_to_be_of_type("Quantity", "int64", mostly=None, 
                                             result_format="BASIC", include_config=True).to_json_dict()

if verification_dict["rule_1_check"]["success"]:
    print("Rule 1 passed")
else:
    print("Rule 1 failed")

Rule 1 passed


#### Rule 2 verification

In [13]:
orders_df["selling_cal"] = orders_df["Quantity"] * orders_df["UnitCost"]
orders_df.selling_cal = orders_df.selling_cal.round()
orders_df["act_selling_round"] = orders_df.SellingCost.round()
verification_dict["rule_2_check"] = orders_df.expect_column_pair_values_to_be_equal("selling_cal", "act_selling_round", mostly=None, 
                                             result_format="BASIC", include_config=True).to_json_dict()

if verification_dict["rule_2_check"]["success"]:
    print("Rule 2 passed")
else:
    print("Rule 2 failed")

Rule 2 failed


#### Rule 3 verification

In [14]:
orders_df["selling_cal"] = orders_df["Quantity"] * orders_df["UnitPrice"]
orders_df.selling_cal = orders_df.selling_cal.round()
orders_df["act_selling_round"] = orders_df.SellingPrice.round()
verification_dict["rule_3_check"] = orders_df.expect_column_pair_values_to_be_equal("selling_cal", "act_selling_round", mostly=None, 
                                             result_format="BASIC", include_config=True).to_json_dict()

if verification_dict["rule_3_check"]["success"]:
    print("Rule 3 passed")
else:
    print("Rule 3 failed")

Rule 3 failed


#### Rule 2,3

Ideally by logic cost * units should give the total cost, however there are some orders where this condition is not matching. We should confirm these condition from client.

Similar goes for Price * units

In [15]:
# Back to Pandas
orders_df = orders_df.drop('selling_cal', 1)
orders_df = orders_df.drop('act_selling_round', 1)
orders_df = pd.DataFrame(orders_df)

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.



### Table cardinality

In [16]:
from  ta_lib.core.api import setanalyse

setanalyse(orders_df.SKU.tolist(),prod_df.SKU.tolist())

{'A-B': 5, 'B-A': 0, 'AuB': 13549, 'A^B': 13544}

This represents a venn diagram on two lists. Left list is `A` & right one is `B`. `A-B` implies that are five SKUs in orders_df missing in product master. We can find them using unsimplied version.

Let us look at the set `A-B`.

In [17]:
missings_master_skus = setanalyse(orders_df.SKU.tolist(),prod_df.SKU.tolist(),simplify=False)['A-B']
missings_master_skus

{'APL AWS3AL(G) 42MM GLD CST',
 'APL AWS3AL(G+C) 38MM GRY CST',
 'SP3 SMS N920 NOTE5 32G GLD DST',
 'SP3 SMS S8+ G955 BLK BER SPT',
 'UNLKD SONY XPERIA XZS BLK 32GB'}

In [18]:
import numpy as np
print(('Records affected due to missing keys are {0} accounting to {1}% of orders').format(
    orders_df.SKU.isin(missings_master_skus).sum(),np.round(orders_df.SKU.isin(missings_master_skus).mean()*100,2)))

Records affected due to missing keys are 402 accounting to 0.23% of orders


**Since missing keys are very less we can proceed with inner join**

**Dev NOTES**

<details>
1. Cardinality with mulitple keys: If you have more than one key use setanalyse_df. You can pass dataframes instead of lists and the key columns
2. Excess master data (if `B-A` >0 in above example) will not be concern
3. If the `A-B` is larger, please check with client for alternative data sources. In case of left join keep a stragey for imputing.


### Master uniqueness

Product master is expected to have non duplicate primary keys. Let us verify them.

In [19]:
# Snippet 
df_freq = prod_df.SKU.value_counts().reset_index()
df_freq.columns = ['SKU','Frequency']
fil_ = df_freq.Frequency>1
if fil_.sum() > 0:
    print(("Found {0} duplicates in master. Sample duplicates are:").format(fil_.sum()))
    print(df_freq[fil_].head())
else:
    print("No duplciates in primary key")

Found 1 duplicates in master. Sample duplicates are:
                              SKU  Frequency
0  UNLKD SONY XPERIA XZS BLUE 32G          2


In [20]:
len(prod_df)

13545

In [21]:
# Dropping inconsistent records
print(("No. of rows before dropping duplicate SKUs: {0}".format(prod_df.shape[0])))
fil_ = (prod_df.SKU == 'UNLKD SONY XPERIA XZS BLUE 32G') & (prod_df.color.str.strip() == 'BLACK')
prod_df = prod_df[~fil_]
print(("No. of rows after dropping duplicate SKUs: {0}".format(prod_df.shape[0])))

No. of rows before dropping duplicate SKUs: 13545
No. of rows after dropping duplicate SKUs: 13544


## Health Analysis

Get an overview of the overall health of the dataset. This is usually quick to compute and hopefully highlights some problems to focus on.



### Summary Plot

Provides a high level summary of the dataset health.

**Watch out for:**

* too few numeric values
* high % of missing values
* high % of duplicate values
* high % of duplicate columns 

In [22]:
sum1, plot1 = eda.get_data_health_summary(orders_df, return_plot=True)
sum2, plot2 = eda.get_data_health_summary(prod_df, return_plot=True)

display_as_tabs([('orders', plot1), ('product', plot2)])

tzname GB identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.



Tabs
    [0] HoloViews(Layout, name='orders')
    [1] HoloViews(Layout, name='product')

**Dev NOTES**

<details>
1. Datatypes : We have both numeric and other types. The bulk of them seem to be numeric. `Numeric` is defined to be one of [float|int|date] and the rest are categorized as `Others`. A column is assumed to have `date` values if it has the string `date` in the column name.

2. The missing value plot seems to indicate missing values are not present but we do have them. 

3. We are looking for duplicate observations (rows in the data). The plot shows the % of rows that are an exact replica of another row (using `df.duplicated`)

4. We are looking for duplicate features (columns in the data).

</details>

### Missing Values summary

This provides an overall view focussing on amount of missing values in the dataset.

**Watch out for:**
* A few columns have significant number of missing values 
* Most columns have significant number of missing values


In [23]:
sum1, plot1 = eda.get_missing_values_summary(orders_df, return_plot=True)
sum2, plot2 = eda.get_missing_values_summary(prod_df, return_plot=True)

display_as_tabs([('orders', plot1), ('product', plot2)])

tzname GB identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.



Tabs
    [0] HoloViews(Bars, name='orders')
    [1] HoloViews(Bars, name='product')

**Dev notes:**

<details>
    
    * By default, the following are considered missing/NA values : `[np.Nan, pd.NaT, 'NA', None]`
    * additional values can be passed to tigerml (add_additional_na_values)
    * these are applied to all columns.
    
    * some of the above information can be learnt from the data discovery step (see discussion below)
    
</details>

In [24]:
sum1 = eda.get_duplicate_columns(orders_df)
sum2 = eda.get_duplicate_columns(prod_df)

display_as_tabs([('orders', sum1), ('product', sum2)])

tzname GB identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.



Tabs
    [0] Markdown(str, name='orders')
    [1] Markdown(str, name='product')

In [25]:
sum1 = eda.get_outliers(orders_df)
sum2 = eda.get_outliers(prod_df)

display_as_tabs([('orders', sum1), ('product', sum2)])

tzname GB identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.



Tabs
    [0] DataFrame(height=300, name='orders', sizing_mode='fixed', value=              ('Data Shape..., width=800)
    [1] Markdown(str, name='product')

## Health Analysis report

Generate a report that has all the above data in a single html. This could be useful to submit to a client

In [26]:
from ta_lib.reports.api import summary_report

summary_report(orders_df, './orders.html')
summary_report(prod_df, './prod.html')

this method is deprecated in favour of `Styler.to_html()`

tzname GB identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.

this method is deprecated in favour of `Styler.to_html()`



prod: https://drive.google.com/file/d/1TM-T5HzAYpT8_1ugM5L6Bnfxp8r3uMem/view?usp=sharing

orders: https://drive.google.com/file/d/1uvehi90v1HFtScZrtWg3pW2zpi-DFQ1Y/view?usp=sharing
    